In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import pymysql.cursors

api_key = '7173ac3049183122f6704ddf9b43425d'

def lat_lon_to_addr(lon, lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=lon, latitude=lat)
    headers = {"Authorization": "KakaoAK " + api_key}
    response = requests.get(url, headers=headers)
    result = response.json()
    if 'documents' in result:
        region = result['documents'][0]['region_2depth_name']
        return region
    else:
        return None

# 데이터셋 로드
dataset = pd.read_excel('data.xlsx')

# 광주 지역 위도 경도 범위 설정
latitude_range = (35.126520, 35.238336)
longitude_range = (126.805994, 126.945419)

# 광주 지역 데이터 필터링
dataset = dataset[
    (dataset['latitude'] >= latitude_range[0]) & (dataset['latitude'] <= latitude_range[1]) &
    (dataset['longitude'] >= longitude_range[0]) & (dataset['longitude'] <= longitude_range[1])
]


# 결측치 제거
dataset = dataset.dropna()


# 위도와 경도를 주소로 변환하여 자치구 정보를 추가
dataset['자치구'] = dataset.apply(lambda row: lat_lon_to_addr(row['longitude'], row['latitude']), axis=1)


# 자치구에 남구, 북구, 서구, 동구, 광산구가 아닌 데이터 삭제
valid_regions = ['남구', '북구', '서구', '동구', '광산구']
dataset = dataset[dataset['자치구'].isin(valid_regions)]

# 구별 인구밀도 정보
인구밀도 = {
    '동구': 2352,
    '서구': 6637,
    '남구': 3478,
    '북구': 3804,
    '광산구': 1415
}

# 구별 인구밀도 정보 추가
dataset['인구밀도'] = dataset['자치구'].map(인구밀도)
dataset.to_excel('inventors.xlsx', index=False)

# 여기까지해서 데이터베이스에 저장


